In [51]:
from neo4j import GraphDatabase
from langchain.document_loaders import WikipediaLoader 
from langchain.text_splitter import TokenTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from yfiles_jupyter_graphs import GraphWidget
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
import os

In [9]:
%reload_ext dotenv
%dotenv

In [10]:
neo4j_url = os.getenv('NEO4J_URI')
neo4j_username = os.getenv('NEO4J_USERNAME')
neo4j_password = os.getenv('NEO4J_PASSWORD')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [11]:
neo4j_password

'WBJWTJMQKF7nqYWX5GVRd2STrGtA70r8RpYV9pWOfx8'

In [12]:
graph = Neo4jGraph(url=neo4j_url, username=neo4j_username, password=neo4j_password)

In [53]:
raw_documents = WikipediaLoader(query="Lionel Messi").load()

In [54]:
raw_documents[:2]

[Document(page_content='Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d\'Or awards, a record six European Golden Shoes, and was named the world\'s best player for a record eight times by FIFA. Until 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles, and the UEFA Champions League four times. With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative playmaker, Messi holds the records for most goals (474), hat-tricks (36), and assists in La Liga (192). He has the most international goals by a South American male (106). Messi has sco

In [55]:
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [56]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
llm_transformer = LLMGraphTransformer(llm=llm)

In [58]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [59]:
graph_documents

[GraphDocument(nodes=[Node(id='Ligue 1', type='Trophy'), Node(id='FIFA World Cup', type='Trophy'), Node(id='Inter Miami', type='Club'), Node(id='Lionel Andrés "Leo" Messi', type='Person'), Node(id='European Golden Shoes', type='Award'), Node(id='Paris Saint-Germain', type='Club'), Node(id='Barcelona', type='Club'), Node(id='Copa del Rey', type='Trophy'), Node(id='Copa América', type='Trophy'), Node(id='Argentina national team', type='Team'), Node(id='Leagues Cup', type='Trophy'), Node(id="FIFA World's Best Player", type='Award'), Node(id="Ballon d'Or", type='Award'), Node(id='UEFA Champions League', type='Trophy'), Node(id='La Liga', type='Trophy')], relationships=[Relationship(source=Node(id='Lionel Andrés "Leo" Messi', type='Person'), target=Node(id='Inter Miami', type='Club'), type='PLAYS_FOR'), Relationship(source=Node(id='Lionel Andrés "Leo" Messi', type='Person'), target=Node(id='Inter Miami', type='Club'), type='CAPTAINS'), Relationship(source=Node(id='Lionel Andrés "Leo" Messi'

In [60]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))